<a href="https://colab.research.google.com/github/kentstephen/duckdb_h3/blob/main/montpelier_water_buildings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize Buildings near Water with H3

### You can hit run all

## First we install dependencies

### Had a little trouble with conflicts between pyarrow and overturemaps. Hopefully it works for you, drope me a line if you have issues.

In [ ]:
! pip install ipywidgets duckdb geopandas shapely pynhd keplergl pandas overturemaps==0.7.0 pyarrow==15.0.2 --quiet

## The values in the cell below are the paramaters you can change.
### If you want to change the `bbox`, you can use [this bounding box tool](https://boundingbox.klokantech.com/), just select csv and paste it in there. The `buffer_distance` is the distance between the buildings and water.
#### Just keep in mind that using the Overture Maps Python downloader like below can be RAM intensive for large areas. Also running large areas with an H3 resolution of 10 will produce a lot of data. And I think this API just works in the US, maybe CONUS

In [2]:
# in meters
buffer_distance = 100
# h3 resolution
resolution = 10
# area of interest
bbox = -72.818472,44.072055,-72.299101,44.41482 # Montpelier and Barre area, Vermont

## Here we query USGS Hydrology data with the Python package `pynhd`.
### I wasn't sure how to construct the request, so I looked at the examples on [HyRiver](https://docs.hyriver.io/examples.html). I recognized one example from [this Fused UDF](https://github.com/fusedio/udfs/tree/main/public/REM_with_HyRiver), that got me up and running.

### So we get the GeoDataFrames passing our `bbox` then we combine them, add our buffer, then reproject to `ESPG:4326` so we can join our Overture Data. Then we have a dataframe.
### Also notice that we are using shapely to turn our `result_df` geometry into `wkt`. One thing I've learned using DuckDB Spatial is that when going to and from Pandas DataFrames, always hand off as text.

In [3]:
import pandas as pd
from shapely import wkt
import geopandas as gpd
import pynhd


# Fetch flowlines and water bodies
wd_flowlines = pynhd.WaterData("nhdflowline_network")
flw_gdf = wd_flowlines.bybox(bbox)

wd_waterbodies = pynhd.WaterData("nhdwaterbody")
water_bodies_gdf = wd_waterbodies.bybox(bbox)

# Combine flowlines and water bodies into one GeoDataFrame
combined_gdf = pd.concat([flw_gdf, water_bodies_gdf], ignore_index=True)

# Convert WKT strings to Shapely geometries if needed and set the CRS
combined_gdf['geometry'] = combined_gdf['geometry'].apply(lambda geom: wkt.loads(geom) if isinstance(geom, str) else geom)
combined_gdf = gpd.GeoDataFrame(combined_gdf, geometry='geometry')

# Check and set CRS if not already set
if combined_gdf.crs is None:
    combined_gdf.set_crs(epsg=4326, inplace=True)

# Convert to EPSG:3857 for buffering
combined_gdf = combined_gdf.to_crs(epsg=3857)

# Apply the buffer
combined_gdf['geometry'] = combined_gdf['geometry'].buffer(buffer_distance)

# Convert back to EPSG:4326
combined_gdf = combined_gdf.to_crs(4326)

# Convert GeoDataFrame back to DataFrame with geometries as WKT
result_df = pd.DataFrame(combined_gdf)
result_df['geometry'] = result_df['geometry'].apply(lambda geom: geom.wkt if geom is not None else None)

result_df


,geometry,comid,fdate,resolution,gnis_id,gnis_name,lengthkm,reachcode,flowdir,wbareacomi,...,shape_area,onoffnet,purpcode,purpdesc,meandepth,lakevolume,maxdepth,meandused,meandcode,lakearea
0,"POLYGON ((-72.30676568345652 44.2957482924395,...",4573763,1999-08-13T04:00:00Z,Medium,1459719,Stillwater Brook,3.590,01080103000118,With Digitized,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POLYGON ((-72.30469544856716 44.22896883140456...,4573765,2008-07-21T04:00:00Z,Medium,1459574,South Branch Wells River,3.491,01080103000124,With Digitized,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POLYGON ((-72.30308675543797 44.20298744533207...,4573785,1999-08-13T04:00:00Z,Medium,1457784,Health Brook,4.376,01080103000126,With Digitized,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POLYGON ((-72.33629690792974 44.19613980616574...,4573801,1999-08-13T04:00:00Z,Medium,1460038,Waits River,9.113,01080104002483,With Digitized,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"POLYGON ((-72.3484397306909 44.14215950956302,...",4573803,1999-08-13T04:00:00Z,Medium,,,3.716,01080104002485,With Digitized,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,POLYGON ((-72.55929760744384 44.08640869141528...,6082677,1999-08-13T04:00:00Z,Medium,,,NaN,01080105000614,NaN,NaN,...,1.789035e-06,1.0,,,1.251247,19920.40388,2.974085,1.251247,4,15920.440892
398,POLYGON ((-72.56026964512579 44.08720152016703...,6082679,1999-08-13T04:00:00Z,Medium,1459675,Staples Pond,NaN,01080105000615,NaN,NaN,...,4.141703e-06,1.0,,,3.860419,142286.13290,8.038214,3.860419,4,36857.693019
399,POLYGON ((-72.73852887424651 44.08332131379535...,6082681,1999-08-13T04:00:00Z,Medium,,,NaN,01080105000621,NaN,NaN,...,1.601551e-06,0.0,,,NaN,NaN,NaN,NaN,None,NaN
400,POLYGON ((-72.73940098286928 44.08237196813772...,6082683,1999-08-13T04:00:00Z,Medium,,,NaN,01080105000622,NaN,NaN,...,7.680006e-07,0.0,,,NaN,NaN,NaN,NaN,None,NaN


## Downloading Overture Data with the Overture Maps Python API. This returns an arrow table we can directly query in DuckDB
### I normally query the Overture S3 Bucket directly, but it's actually faster for me to do that with larger areas. I will share another example of how to do that, by querying Overture's Land Division Area theme.

In [4]:
import overturemaps
overture = overturemaps.record_batch_reader("building", bbox).read_all()


## You can actually load h3 by writing this now ```INSTALL h3 FROM community; LOAD h3;``` but when I tested this just now on Google Colab it would not let me.

In [5]:
import duckdb
con = duckdb.connect(config={"allow_unsigned_extensions": True})
con.sql(""" INSTALL h3ext FROM 'https://pub-cc26a6fd5d8240078bd0c2e0623393a5.r2.dev';
            LOAD h3ext;
            INSTALL spatial;
            LOAD spatial;
            INSTALL httpfs;
            LOAD httpfs;
            SET s3_region='us-west-2';""")

## This is to get the building points into H3 cells. First we get the geometry from the arrow table, cut it up into centroids. Then we make a grid and add a `COUNT(1)` to get the volume of buildings in each cell. Look at [this page](https://github.com/isaacbrodsky/h3-duckdb) to see the bindings for the extension

In [6]:

query = """
    CREATE OR REPLACE TABLE buildings AS

    WITH geometry_cte AS (
        SELECT
            id,
            ST_GeomFromWKB(geometry) as geometry
        FROM overture
    ),
    centroids as (
         SELECT
            id,
            ST_Y(ST_Centroid(geometry)) AS latitude,
            ST_X(ST_Centroid(geometry)) AS longitude
        FROM geometry_cte
    ),
    h3_cells_cte AS (
        SELECT
            id,
            h3_latlng_to_cell(latitude, longitude, $resolution) AS cell_id,
            COUNT(1) as cnt
        FROM centroids
        GROUP BY id, h3_latlng_to_cell(latitude, longitude, $resolution)
    ),
    h3_agg_cte AS (
        SELECT
            cell_id,
            SUM(cnt) AS total_cnt
        FROM h3_cells_cte
        GROUP BY cell_id
    )
    SELECT
        cell_id,
        total_cnt
    FROM h3_agg_cte
"""

# Create a dictionary with the required keys and values
params = {'resolution': resolution,}

# Execute the query with the parameters
con.sql(query=query, params=params)

# AND ST_Intersects(ST_GeomFromText($wkt_geom), ST_GeomFromWKB(geometry))


## Now we join our buffer we created earlier and see where water is compared to buildings.

In [7]:
query = """
--CREATE OR REPLACE TABLE vermont AS

WITH w_buffer AS (
    SELECT
        ST_GeomFromText(geometry) AS water_buffer
    FROM result_df
)
SELECT

    h3_h3_to_string(b.cell_id) AS cell_id,
    h3_cell_to_boundary_wkt(b.cell_id) AS cell_boundary,
    b.total_cnt
FROM buildings b
JOIN w_buffer w
ON ST_Intersects(ST_GeomFromText(h3_cell_to_boundary_wkt(b.cell_id)), w.water_buffer)
GROUP BY b.cell_id, b.total_cnt;
"""
df_b = con.sql(query).df()
df_b

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cell_id,cell_boundary,total_cnt
0,8a2bab0cc79ffff,"POLYGON ((-72.418657 44.415270, -72.419648 44....",1.0
1,8a2bab0ceb4ffff,"POLYGON ((-72.421629 44.414844, -72.422619 44....",5.0
2,8a2bab0da687fff,"POLYGON ((-72.549165 44.411794, -72.550155 44....",2.0
3,8a2bab0ce657fff,"POLYGON ((-72.446715 44.410634, -72.447706 44....",4.0
4,8a2bab0ce207fff,"POLYGON ((-72.435809 44.410363, -72.436800 44....",3.0
...,...,...,...
3553,8a2bab2c68b7fff,"POLYGON ((-72.335311 44.080253, -72.336295 44....",1.0
3554,8a2bab2c6c6ffff,"POLYGON ((-72.337929 44.078659, -72.338912 44....",1.0
3555,8a2b8d91290ffff,"POLYGON ((-72.649212 44.076919, -72.650195 44....",1.0
3556,8a2b8d910ceffff,"POLYGON ((-72.631806 44.075189, -72.632788 44....",2.0


## We could probably plot directly from the DF or GDF but just so we're consistent. Reminder that the geometry comes in as text and is being loaded into the DataFrame as text

In [8]:
query = """
      SELECT
        geometry
      FROM result_df
"""
df_w = con.sql(query).df()
df_w

,geometry
0,"POLYGON ((-72.30676568345652 44.2957482924395,..."
1,POLYGON ((-72.30469544856716 44.22896883140456...
2,POLYGON ((-72.30308675543797 44.20298744533207...
3,POLYGON ((-72.33629690792974 44.19613980616574...
4,"POLYGON ((-72.3484397306909 44.14215950956302,..."
...,...
397,POLYGON ((-72.55929760744384 44.08640869141528...
398,POLYGON ((-72.56026964512579 44.08720152016703...
399,POLYGON ((-72.73852887424651 44.08332131379535...
400,POLYGON ((-72.73940098286928 44.08237196813772...


## Config from Kepler to load my map

In [9]:
kepler_config = {'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '0csmpkq',
     'type': 'geojson',
     'config': {'dataId': 'buildings',
      'label': 'buildings',
      'color': [198, 198, 198],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'cell_boundary'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [221, 178, 124],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': True,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': {'name': 'total_cnt', 'type': 'integer'},
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}},
    {'id': '4utcft',
     'type': 'geojson',
     'config': {'dataId': 'water',
      'label': 'water',
      'color': [84, 56, 200],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [255, 153, 31],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'enableElevationZoomFactor': True,
       'stroked': False,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'buildings': [{'name': 'total_cnt',
        'format': None}],
      'water': []},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 24,
   'dragRotate': True,
   'latitude': 44.24400392025825,
   'longitude': -72.58721037341296,
   'pitch': 50,
   'zoom': 9.973237340305365,
   'isSplit': False},
  'mapStyle': {'styleType': 'muted',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [224.4071295378559,
    224.4071295378559,
    224.4071295378559],
   'mapStyles': {}}}}


## Google Colab needs to import a widget manager so this will do that if you are using it or it will ignore if you are not

In [10]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    print("Not running in Google Colab. Skipping custom widget manager setup.")

Not running in Google Colab. Skipping custom widget manager setup.


## And here you go. Again, to change the way the map looks go back to the top of the notebook and adjust the parameters.

### The height comes from the number of buildings in that cell

In [11]:
# Load an empty map
from keplergl import KeplerGl

# Create an instance of KeplerGl
map_1 = KeplerGl()

# Add your data to the map
map_1.add_data(data=df_b, name='buildings')
map_1.add_data(data=df_w, name='water')
map_1.config = kepler_config
# Display the map
map_1


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '0csmpkq', 'type': …

In [12]:
# map_1.config